In [ ]:
from scripts.readers import read_images, read_image, save_TIFF

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import porespy as ps
import importlib as il
import os

from scipy.stats import wilcoxon, ks_2samp, f_oneway
from skimage.io import imread # read in images
from skimage.measure import regionprops, regionprops_table, shannon_entropy
from skimage.metrics import hausdorff_distance
from sklearn.metrics import roc_curve, auc, jaccard_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from collections import OrderedDict
from importlib.metadata import version

In [ ]:
segment_kmeans_clust = il.import_module("segment.kmeans_clust")
segment_extract_pores = il.import_module("segment.extract_pores")
segment_process_segments = il.import_module("segment.process_segments")
segment_distance_thickness = il.import_module("segment.distance_thickness")

pores_pore_network = il.import_module("pores.pore_network")
pores_pore_regionprops = il.import_module("pores.pore_regionprops")

plots_plots_3d = il.reload(plots_plots_3d)
plots_colormaps_lec = il.reload(plots_colormaps_lec)

In [ ]:
segment_kmeans_clust = il.reload(segment_kmeans_clust)
segment_extract_pores = il.reload(segment_extract_pores)
segment_process_segments = il.reload(segment_process_segments)
segment_distance_thickness = il.reload(segment_distance_thickness)

pores_pore_network = il.reload(pores_pore_network)
pores_pore_regionprops = il.reload(pores_pore_regionprops)

plots_plots_3d = il.reload(plots_plots_3d)
plots_colormaps_lec = il.reload(plots_colormaps_lec)

In [ ]:
# NOTE: Need to implement False
load_single = True # If False, will load all specified bread samples as a list

In [ ]:
vx_size = 60.9e-6 # Voxel size in metres

if load_single:
    # start_indices = [150, 299, 180, 300]; start_indices2 = [1420, 1440, 1420, 1440]
    # end_indices = [799, 853, 829, 899]; end_indices2 = [2069, 1994, 2069, 2039]
    # bread_types = ["Ro1a", "Ru1a", "V1a", "W1a"]
    
    start_idx = 180
    end_idx = 829
    bread_type = 'V1a'
    #save_slice_num = str(int(end_idx-start_idx+1))
    # FIX: automatic slicenum detection
    if bread_type == 'Ro1a':
        save_slice_num = "650"
    elif bread_type == 'Ru1a':
        save_slice_num = "555"
    elif bread_type == 'V1a':
        save_slice_num = "650"
    elif bread_type == 'W1a':
        save_slice_num = "600"
    else:
        print('Check Bread Type')
    output_path = '/cluster/scratch/spruthi/qbi2024_project/'+bread_type+'_save_'+save_slice_num
    if os.path.exists(output_path):
        print('Starting')
    else:
        print('Target path created, check '+output_path)
        os.makedirs(output_path, exist_ok=True)
        # FIX: Need to fix the path
        print('You may want to manually specify save_slice_num')

In [ ]:
if load_single:
    # Load stored images

    # Bread structure (pores = 0)
    connthresh_medfilter3_ds_img_1 = read_images(os.path.join(output_path, (bread_type+'_connthresh_medfilter3_ds_img_1_{0:04}.tif')), start_idx, end_idx)
    print(connthresh_medfilter3_ds_img_1.shape)

    # Filled bread (solid structure)
    filled_pores_1 = read_images(os.path.join(output_path, (bread_type+'_filled_pores_1_{0:04}.tif')), start_idx, end_idx)

    # Pores segmented based on connected components labelling
    merged_labelled_image_lbl = read_images(os.path.join(output_path, (bread_type+'_merged_labelled_image_lbl_{0:04}.tif')), start_idx, end_idx)

    # Pores segmented based on watershed analysis
    merged_labelled_image_ws = read_images(os.path.join(output_path, (bread_type+'_merged_labelled_image_ws_{0:04}.tif')), start_idx, end_idx)

    # Local thicnkess map of pores generated using porespy
    lt_ws = read_images(os.path.join(output_path, (bread_type+'_lt_ws_{0:04}.tif')), start_idx, end_idx)
    
    # Inverted image - extracted pores (if exists)
    if os.path.exists(os.path.join(output_path, (bread_type+'_inverted_image_1_{start_idx}.tif'))):
        inverted_image_1 = read_images(os.path.join(output_path, (bread_type+'_inverted_image_1_{0:04}.tif')), start_idx, end_idx)
        print('Inverted image loaded')
        
    # Loading metrics (if needed)
    #df_pair_pore = pd.read_pickle(os.path.join(output_path, (bread_type+'_df_pair_pore.pkl')))
    #df_pair_pore2 = pd.read_pickle(os.path.join(output_path, (bread_type+'_df_pair_pore2.pkl')))

In [ ]:
# Compare segmentations - Jaccard index, Dice, Hausdorff distance
# Radial density distribution (and pore size distribution)
# Pipeline at a glance
# Montage pad - show
# 3d plot of image volume
# Network - porespy and custom

In [ ]:
# connthresh_medfilter3_ds_img_1
# filled_pores_1
# inverted_image_1
# merged_labelled_image_lbl
# merged_labelled_image_ws
# lt_ws